https://pytorch.org/tutorials/beginner/nn_tutorial.html

https://github.com/spro/practical-pytorch/tree/master/char-rnn-classification

https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

Batch RNN
https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html


![alt text](https://docs.gdc.cancer.gov/Encyclopedia/pages/images/barcode.png "TCGA Barcode")

To identify a sample, which is used for both methylation and expression measurements, we need only first 4 "-" separated identifiers, i.e. up to sample:vial level;

It is assumed that each patient is measured once (valid).

In [1]:
import re
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
torch.manual_seed(42)


from pathlib import Path
import pandas as pd

DIR = Path("/data/eugen/meth2expr/")
TCGA_DIR = Path("/data/eugen/tcga/projects")

projects = [str(x).split("/")[-1] for x in TCGA_DIR.iterdir() if x.is_dir()]
projects_paths = [x for x in TCGA_DIR.iterdir() if x.is_dir()]



#print(torch.cuda.is_available())
#if torch.cuda.is_available() or True:
#    device = torch.device("cuda")          # a CUDA device object
#    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
#    x = x.to(device)                       # or just use strings ``.to("cuda")``
#    z = x + y
#    print(z)
#    print(z.to("cpu", torch.double))   

ImportError: TensorBoard logging requires TensorBoard with Python summary writer installed. This should be available in 1.14 or above.

In [ ]:
PROJECTS_DIR = "/data/eugen/tcga/projects/"

class Project(object):
    """
    Project data loader and descriptor.
    
    :attributes:
    
    :public:
        get_case_data()
    
    :private:
        _collect_samples()
        _collect_metadata()
        _get_case_datapaths()
    """
    
    def __init__(self, name, projects_dir=PROJECTS_DIR):
        '''
        :params
            name -- project name
            path -- project dir
        '''
        self.name = name
        self.dir = projects_dir + name
        
        self.meth_path = None
        self.meth_fpath = None
        
        self.samples = {}
        self.sample_ids = []
        self.sample_paths = {}
        self.cases = {}
        self.case_ids = []
        self.meta = {}
        
        # Get samples ids and paths
        self._collect_samples()
        self._collect_metadata()
        
        
    def _collect_samples(self):
        """
        Extracts samples' ids and paths from project directory.
        """
        
        self.sample_paths = {"methylation": {},
                              "expression": {}}
        
        # Extract methylation
        self.meth_path = Path(self.dir) / 'data/methylation' / self.name / 'harmonized/DNA_Methylation/Methylation_Beta_Value'
        self.expr_path = Path(self.dir) / 'data/expression' / self.name / 'harmonized/Transcriptome_Profiling/Gene_Expression_Quantification'
        self.sample_paths = {datatype: {str(f).split("/")[-1]: str(f) 
                                        for f in path.iterdir()}
                             for datatype, path in [('methylation', self.meth_path), ('expression', self.expr_path)]}
        
        # In path there is an experiment ID, not sample id
        self.samples = {f: list(self.sample_paths[f].keys()) for f in ['methylation', 'expression']}
        self.sample_ids = list(self.samples.keys())
    
    
    def _collect_metadata(self):
        """
        Gets all necessary metadata of project.
        Includes ids and case ids: necessary to match expression and methylation measurements.
        """
        
        metadata_paths = [(Path(self.dir) / (self.name + "_" + f + ".csv"))
                              for f in ['methylation', 'expression'] ]
        
        ### Access metadata files
        cols = ['file_id', 'file_name', 'cases']
        meth, expr = list(map(lambda p: pd.read_csv(p, sep='\t', usecols=cols), metadata_paths))
        
        
        ### Prune the TCGA barcode of case ids up to sample level, resulting in Project-TSS-ParticipantID-<SampleID><vial>
        barcode_splitter = lambda x: "-".join(x.split('-')[:4])
        meth['cases'] = meth['cases'].apply(barcode_splitter)
        expr['cases'] = expr['cases'].apply(barcode_splitter)

                          
        meth = meth.set_index("file_id", drop=False)
        expr = expr.set_index("file_id", drop=False)
        
        meta = {'methylation': meth.to_dict(orient='index'),
                'expression': expr.to_dict(orient='index') }
        
        # Assert equivalency of ids in metadata files and available files
        assert set(meta['methylation'].keys()) == set(self.samples['methylation'])
        assert set(meta['expression'].keys()) == set(self.samples['expression'])
        
        self.meta = meta
        
        
        ### Collect metadata for cases
        meth = meth.set_index("cases")
        expr = expr.set_index("cases")
        
        cases = {'methylation': meth.to_dict(orient='index'),
                 'expression': expr.to_dict(orient='index') }
        self.old_cases = cases
        # recheck equivalency of cases for methylation and expression (should be true by constraint of download script)
        assert set(cases['methylation'].keys()) == set(cases['expression'].keys())
        
        # make cases as keys { case -> {methylation -> {}; expression  -> {}} }
        new_dict = {case: {'methylation': cases['methylation'][case],
                           'expression': cases['expression'][case]}
                    for case in cases['methylation'].keys() }
        
        self.cases = new_dict
        self.case_ids = list(self.cases.keys())
        
        
    def _get_case_datapaths(self, case: str):
        '''Getter method for meth, expr or both types data.'''
        
        assert case in self.cases.keys()
        
        self.meth_fpath = Path(self.meth_path) / self.cases[case]['methylation']['file_id'] / self.cases[case]['methylation']['file_name']
        self.expr_fpath = Path(self.expr_path) / self.cases[case]['expression']['file_id'] / self.cases[case]['expression']['file_name']
        
        return {'methylation': self.meth_fpath, 'expression': self.expr_fpath}
    
    
    def get_case_data(self, case: str, genes = None, cpgs = None) -> dict:
        """
        Get case methylation and expression dataframe.
        
        :params
            case -- case id;
            genes -- list of genes to get data for. if None: gets data for all genes;
            cpgs -- list of cpgs to get data for. if None: gets data for all cpgs;
        """
        
        get_expr, get_meth = True, True
        paths = self._get_case_datapaths(case)
        
        # We ignore rest columns, since terribly redundant. They describe CpG features, which are assumed to be constant for every
        # experiment (to be tested). That's the reason for the terribly inefficient data storage. After testing invariance,
        # methylation data files have to cleared (i.e. remove redundant columns).
        meth_cols = ['Beta_value']  # 'Composite Element REF' becomes index
        
        # load both meth and expr for now
        # if dtype == 'methylation':
        #     get_expr = False
        # elif dtype == 'expression':
        #     get_meth = False
        
        if get_expr:
            if genes is not None:
                expr = pd.read_csv(paths['expression'], sep='\t', index_col=0, header=None).loc[genes]
            else:
                expr = pd.read_csv(paths['expression'], sep='\t', index_col=0, header=None)
        
        if get_meth:
            if cpgs is not None:
                meth = pd.read_csv(paths['methylation'], sep='\t', index_col=0, header=0)[meth_cols].loc[cpgs]
            else:
                meth = pd.read_csv(paths['methylation'], sep='\t', index_col=0, header=0)[meth_cols]
        
        return {'methylation': meth, 'expression': expr}

In [ ]:
def base2tensor(base: str, vocab: list = VOCAB) -> torch.tensor:
    tensor = torch.zeros(1, len(vocab))
    tensor[BASE2IDX[base]][0] = 1
    return tensor
    
def seq2tensor(seq: str, vocab: list = VOCAB) -> torch.tensor:
    tensor = torch.zeros(len(seq), 1, len(vocab))  # important choice for preserving shape compatibility with hidden layer.
    for idx, base in enumerate(seq):
        tensor[idx][0][BASE2IDX[base]] = 1  # that extra 1 dimension is because PyTorch assumes everything is in batches - we’re just using a batch size of 1 here.
    return tensor

def methylate_seq(seq_tensor: torch.tensor, loc: int, value: float, meth_idx: int, mask_idx: int) -> torch.tensor:
    assert (seq_tensor[loc][0][meth_idx] > 0) or (seq_tensor[loc][0][mask_idx] > 0) # assert it is a "C" in first place
    seq_tensor[loc][0][meth_idx] = value
    return seq_tensor

#################

class Gene(object):
    """
    .
    """
    
    def __init__(self, name, genomics_dir):
        """
        TODO: think about how to compute for each gene local coordinates for CpGs within promoter.
        """
        self.dir = genomics_dir
        self.name = name
        self.cpgs = []
        self.promoter_seq = None  # promoter sequence? or coordinates, etc.
        self.tensor = None
        
        self.mask_idx = -1
        self.meth_idx = -1
        
    def get_promoter_seq(self):
        """
        Load the promoter sequence.
        """
        raise NotImplementedError
    
    def get_cpgs(self):
        """
        Load the gene's cpgs.
        """
        raise NotImplementedError
    
    def tensorize(self):
        """
        Represent sequence as tensor.
        """
        t = seq2tensor(self.promoter_seq)
        self.tensor = t.copy()
        return t
    
    def methylate_tensor(self, cpgs):
        """
        Methylate the tensor with values.
        TODO: think how to interface Gene - Cpg - Dataset...
        """
        assert self.tensor not None
        
        for cpg in cpgs:
            # insert cpg value in row for meth_base (cytosine)
            cpg_loc = cpg[0]
            cpg_value = cpg[1]
            seq = methylate_seq(self.tensor, cpg_loc, cpg_value, self.meth_idx, self.mask_idx)
                    
        self.tensor = seq
        return seq
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.tensor
    
    
g = Gene("ENS000000")
g.name, g.promoter_seq

In [ ]:
'''
Key data loading functionality should be implemented in a dataset object and look like this:

get_data(projects: list, genes: list):
    return (seq, cpgs, expr)
    
Regarding data flow:
1) Create all CpG objects from Illumina file
2) Create all gene objects (here define cpg appartenence to gene)
'''


class Cpg(object):
    """
    .
    """
    
    def __init__(self, name):
        """
        .
        """
        self.name = name
        self.start = None
        self.end = None
        self.strand = None
        self.genes = []

        
class PromoterDataset(Dataset):
    """
    Model an immutable container for dataset.
    """
    
    def __init__(self, projects: list, genes: list):
        '''Initializes dataset with given projects and genes.'''
        
        self.projects = []
        for proj_name in projects:
            # Add each project as Project object, which points to all relevant files for loading records
            # in __get__item
            self.projects.append(Project(proj_name))
        
        self.genes = []
    
    def __len__(self):
        return len(self.projects) * len(self.genes)
    
    def __getitem__(self):
        return 0

In [208]:
# import unittest

# class TestProject(unittest.TestCase):
#     """."""
    
#     self.case = 'TCGA-2G-AAHC-01A'
#     self.genes = ['ENSG00000242268.2', "ENSG00000167578.15", "ENSG00000105063.17"]
#     self.cpgs = ['cg00000029', 'cg00000165', 'rs966367', 'rs9363764']
    
#     def test_get_case_data(self):
#         raise NotImplementedError

In [210]:
proj = 'TCGA-TGCT'
proj_path = '/data/eugen/tcga/projects/TCGA-TGCT'

P = Project(proj, proj_path)

case = 'TCGA-2G-AAHC-01A'
genes = ['ENSG00000242268.2', "ENSG00000167578.15", "ENSG00000105063.17"]
cpgs = ['cg00000029', 'cg00000165', 'rs966367', 'rs9363764']

case_data = P.get_case_data(case, genes, cpgs)
expr = case_data['expression']
meth = case_data['methylation']

In [211]:
meth

,Beta_value
Composite Element REF,
cg00000029,0.374425
cg00000165,0.099877
rs966367,0.526726
rs9363764,0.487821


In [108]:
import pandas as pd
#p = Path("/data/eugen/tcga/projects") / proj
#[f for f in p.iterdir()]
meth = pd.read_csv(Path("/data/eugen/tcga/projects") / proj /"TCGA-TGCT_methylation.csv", sep='\t')
expr = pd.read_csv(Path("/data/eugen/tcga/projects") / proj /"TCGA-TGCT_expression.csv", sep='\t')

In [143]:
#meth = meth.set_index("file_id")
#meth = meth.to_dict(orient='index')
# meth.set_index('cases', drop=False)

,data_type,file_name,cases,id
cases,,,,
TCGA-2G-AAHC-01A-11D-A42Z-05,Methylation Beta Value,jhu-usc.edu_TGCT.HumanMethylation450.1.lvl-3.T...,TCGA-2G-AAHC-01A-11D-A42Z-05,016d83e0-259e-45c4-8bd3-5c784ac459fd
TCGA-2G-AAGT-01A-11D-A42Z-05,Methylation Beta Value,jhu-usc.edu_TGCT.HumanMethylation450.1.lvl-3.T...,TCGA-2G-AAGT-01A-11D-A42Z-05,48862052-ac4d-4691-bfb9-d244e5681f90
TCGA-2G-AALY-01A-11D-A42Z-05,Methylation Beta Value,jhu-usc.edu_TGCT.HumanMethylation450.1.lvl-3.T...,TCGA-2G-AALY-01A-11D-A42Z-05,23760992-0067-467b-9ee3-843056ff60fb
TCGA-2G-AAH8-01A-11D-A42Z-05,Methylation Beta Value,jhu-usc.edu_TGCT.HumanMethylation450.1.lvl-3.T...,TCGA-2G-AAH8-01A-11D-A42Z-05,b454b4f8-1822-4bd4-8c36-752c4fbbd3c8
TCGA-2G-AAGS-01A-11D-A42Z-05,Methylation Beta Value,jhu-usc.edu_TGCT.HumanMethylation450.1.lvl-3.T...,TCGA-2G-AAGS-01A-11D-A42Z-05,5c06ef79-0993-43af-9613-c08e82775b35
...,...,...,...,...
TCGA-2G-AAG9-01A-11D-A42Z-05,Methylation Beta Value,jhu-usc.edu_TGCT.HumanMethylation450.1.lvl-3.T...,TCGA-2G-AAG9-01A-11D-A42Z-05,dd66c667-b77c-4516-93d3-36f576ed5a2c
TCGA-2G-AAGW-01A-11D-A42Z-05,Methylation Beta Value,jhu-usc.edu_TGCT.HumanMethylation450.1.lvl-3.T...,TCGA-2G-AAGW-01A-11D-A42Z-05,49296673-6dfb-45ea-8d02-9bee12a0c64c
TCGA-2G-AAH2-01A-11D-A42Z-05,Methylation Beta Value,jhu-usc.edu_TGCT.HumanMethylation450.1.lvl-3.T...,TCGA-2G-AAH2-01A-11D-A42Z-05,2af135ce-8d51-465d-bef1-7279bfa3c2ee


In [65]:
cols = ['file_id', 'file_name', 'cases']
expr[cols].head()

,file_id,data_type,file_name,cases,id
0,6ef0f618-69e1-4707-904c-9f6f3d7f25cf,Gene Expression Quantification,3e470f27-ee45-4b57-8b84-2ac4b055dcbc.FPKM-UQ.t...,TCGA-2G-AAHC-01A-11R-A430-07,6ef0f618-69e1-4707-904c-9f6f3d7f25cf
1,f95eb680-d99e-4145-b2dd-651311f38acc,Gene Expression Quantification,8c06cbb3-f46e-41b8-8217-bb22ac1b471a.FPKM-UQ.t...,TCGA-2G-AAGT-01A-11R-A430-07,f95eb680-d99e-4145-b2dd-651311f38acc
2,60a9f839-bc01-4705-8c0a-7a2adf9686f6,Gene Expression Quantification,0e6e268d-d830-40ed-95a7-38f4ae1e7eac.FPKM-UQ.t...,TCGA-2G-AALY-01A-11R-A430-07,60a9f839-bc01-4705-8c0a-7a2adf9686f6
3,27e5ccb2-6f5c-4cbc-89e3-28e3f854ce00,Gene Expression Quantification,7868cab1-0300-493b-b910-6208a3c7921a.FPKM-UQ.t...,TCGA-2G-AAH8-01A-11R-A430-07,27e5ccb2-6f5c-4cbc-89e3-28e3f854ce00
4,4b3b4c23-7974-4095-a491-f43fa9d982a7,Gene Expression Quantification,b6cf2f29-6ea4-422e-bfa6-7bbc3e07e0b5.FPKM-UQ.t...,TCGA-2G-AAGS-01A-11R-A430-07,4b3b4c23-7974-4095-a491-f43fa9d982a7


In [72]:
P = Project('TCGA-TGCT', Path("/data/eugen/tcga/projects") / proj )
P.samples['expression']
"f95eb680-d99e-4145-b2dd-651311f38acc" in P.samples['expression']

True

In [84]:
VOCAB = ['A', 'C', 'T', 'G', 'N']
BASE2IDX = {"A": 0, "C": 1, "T": 2, "G": 3, "N": 4}


def findall(base: str, seq: str):
    idxs = []
    for m in re.finditer(base, seq):
        idxs.append(m.start(0))
    return idxs


def random_seq(l: int, vocab: list = VOCAB) -> str:
    return "".join(np.random.choice(vocab, l))


def random_data(m: int, l: int, c: int) -> tuple:
    """
    Generates random dataset consisting of m sequences of length l with up to c cpg sites per sequence.
    """
    assert c < l

    seqs = [random_seq(l, ) for _ in range(m)]
    cpgs = [[(np.random.choice(findall("C", seqs[i])), np.random.random()) for _ in range(np.random.randint(0, c))] for i in range(m)]
    exprs = [float(np.random.binomial(n=1, p=x.count('N')/len(x), size=1)[0]) for x in seqs]

    return seqs, cpgs, exprs


def base2tensor(base: str, vocab: list = VOCAB) -> torch.tensor:
    tensor = torch.zeros(1, len(vocab))
    tensor[BASE2IDX[base]][0] = 1
    return tensor


def seq2tensor(seq: str, vocab: list = VOCAB) -> torch.tensor:
    tensor = torch.zeros(len(seq), 1, len(vocab))  # important choice for preserving shape compatibility with hidden layer.
    for idx, base in enumerate(seq):
        tensor[idx][0][BASE2IDX[base]] = 1  # that extra 1 dimension is because PyTorch assumes everything is in batches - we’re just using a batch size of 1 here.
    return tensor


def methylate_seq(seq_tensor: torch.tensor, loc: int, value: float, meth_idx: int, mask_idx: int) -> torch.tensor:
    assert (seq_tensor[loc][0][meth_idx] > 0) or (seq_tensor[loc][0][mask_idx] > 0)# assert it is a "C" in first place
    seq_tensor[loc][0][meth_idx] = value
    return seq_tensor


class PromoterArtificialDataset(Dataset):
    """
    Dataset class for loading data. Communicates with loader object to bring batched data.
    
    @attributes
    @methods
    
    Example:
    ...
    """

    def __init__(self, data: tuple):
        
        self.seqs, self.cpgs, self.exprs = data
        
        # get some utile attributes
        self.vocab = sorted(set("".join(self.seqs)))  # returns list
        self.vocab_size = len(self.vocab)
        self.base_to_idx = {}
        for i, base in enumerate(VOCAB):
            self.base_to_idx[base] = i
        
        self.meth_base = "C"  #set methylated base
        self.mask_base = "N"
        self.meth_idx = self.base_to_idx[self.meth_base]
        self.mask_idx = self.base_to_idx[self.mask_base]
        
        
    def __len__(self):
        return len(self.exprs)


    def __getitem__(self, idx):
        """Getter."""
        seq = seq2tensor(self.seqs[idx])
        cpgs = self.cpgs[idx]
        expr = self.exprs[idx]

        for cpg in cpgs:
            # insert cpg value in row for meth_base (cytosine)
            cpg_loc = cpg[0]
            cpg_value = cpg[1]
            seq = methylate_seq(seq, cpg_loc, cpg_value, self.meth_idx, self.mask_idx)

        return seq, expr
            

m = 100
l = 50
c = 5
VOCAB = ['A', 'C', 'T', 'G', 'N']
BASE2IDX = {"A": 0, "C": 1, "T": 2, "G": 3, "N": 4}
data = random_data(m, l, c)

# test_loader, train_loader
train_set = PromoterArtificialDataset(data)
train_loader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=0)

In [152]:
class RNNRegressor(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNRegressor, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.i2h = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.i2o = nn.Linear(input_dim + hidden_dim, output_dim)
        self.tanh = nn.Tanh()
    
    def forward(self, input, hidden):
        
        #input = input.view(-1, input.shape[1])
        # TODO: Modify code to accept batch tensors <loc, batch_nr, index>. Can add init hidden in here.
        
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.tanh(output)

        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_dim)

HIDDEN_DIM = 10
rnn = RNNRegressor(train_set.vocab_size, HIDDEN_DIM, 1)



writer = SummaryWriter('runs/RNN_playground')
writer.add_graph(rnn, (seq2tensor("ACGTT")[1], torch.zeros(1, HIDDEN_DIM)))
writer.close()

In [153]:
criterion = nn.MSELoss()
optimizer = optim.SGD(rnn.parameters(), lr=0.005)


def train(output_tensor, seq_tensor):
    ''''''
    
    hidden = rnn.initHidden()
    
    # zero the grad buffers
    rnn.zero_grad()
    optimizer.zero_grad()
    
    # forward pass
    for i in range(seq_tensor.shape[0]):
        output, hidden = rnn(seq_tensor[i], hidden)
    
    # compute loss and backward pass
    loss = criterion(output, output_tensor)
    loss.backward()
    
    # update params
    optimizer.step()
    
    return output, loss.item()

train(torch.tensor([-0.0117]), seq2tensor("ACGTN"))
print(seq2tensor("ACGTN").shape, torch.tensor([-0.0118]).shape)

torch.Size([5, 1, 5]) torch.Size([1])


In [156]:
NUM_EPOCHS = 5

# TODO: not really minibatch for now
for epoch in range(NUM_EPOCHS):
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        # get inputs
        # TODO: Account for dtypes, otherwise get incompatible!
        seq_tensor, expr = data
        seq_tensor = seq_tensor[0]
        expr = expr[0].view(1)
        expr = expr.type(torch.FloatTensor)  
        
        # train on example
        output, loss = train(expr[0], seq_tensor)
        
        # print statistics
        running_loss += loss
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# save model
PATH = './models/test_model'
torch.save(rnn.state_dict(), PATH)

Finished Training


In [ ]:
# Predict loop

# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data
#         outputs = net(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %d %%' % (
#     100 * correct / total))